In [1]:
import csv
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime

In [2]:
train_data = pd.read_csv("data/train.csv")

In [3]:
train_data["url_count"] = train_data["urls"].apply(lambda s: s[1:-1].count("\'")/2)
train_data["text_len"] = train_data["text"].apply(lambda s: len(s))
train_data["hashtags_count"] = train_data["hashtags"].apply(lambda s: s[1:-1].count("\'")/2)
train_data["day"] = train_data["timestamp"].apply(lambda t: datetime.utcfromtimestamp(t/1000).day)
train_data["hour"] = train_data["timestamp"].apply(lambda t: datetime.utcfromtimestamp(t/1000).hour)
train_data["Macron"] =  train_data["text"].apply(lambda s: ("macron" in s.lower().split()))
train_data["Zemmour"] =  train_data["text"].apply(lambda s: ("zemmour" in s.lower().split()))
train_data["Melenchon"] =  train_data["text"].apply(lambda s: ("melenchon" in s.replace("é","e").lower().split()))
train_data["rt"] =  train_data["text"].apply(lambda s: ("rt" in s.lower().split()))

In [4]:
# TODO: include sent analysis

In [5]:
train_data_filtered = train_data.loc[:, ["retweets_count","favorites_count","followers_count","statuses_count","friends_count",
                                 "hashtags_count","day","hour","verified","url_count","text_len","Macron","Zemmour","rt"]]

In [6]:
# Standardize the data
normal_columns = ["favorites_count","followers_count","statuses_count","friends_count","text_len"]
mu, sigma = train_data_filtered[normal_columns].mean(axis=0), train_data_filtered[normal_columns].std(axis=0)
train_data_filtered.loc[:, normal_columns] = (train_data_filtered[normal_columns] - mu) / sigma

In [7]:
import torch
import pytorch_lightning as pl
from setup_data import TweetsDataset
from pytorch_lightning.loggers import TensorBoardLogger
from torch.utils.data import DataLoader, random_split
from mlp import MLP

device = torch.device("cpu")

In [8]:
logger = TensorBoardLogger("lightning_logs", name="model_linear")

In [9]:
model = MLP(h=20, d=0)

In [10]:
# dataset = TweetsDataset(train_data_filtered)

# # Split into training and test
# train_size = int(0.8 * len(dataset))
# test_size = len(dataset) - train_size
# trainset, testset = random_split(dataset, [train_size, test_size])

# # # Dataloaders
# trainloader = DataLoader(trainset, batch_size=32, shuffle=True, num_workers=12)
# testloader = DataLoader(testset, batch_size=32, shuffle=False, num_workers=12)

# # train(train_data_filtered, 3)
# trainer = pl.Trainer(logger=logger, default_root_dir="checkpoints/", limit_train_batches=100, max_epochs=50)
# trainer.fit(model=model, train_dataloaders=trainloader, val_dataloaders=testloader)
# trainer.save

In [11]:
from pytorch_lightning.callbacks import LearningRateMonitor

In [12]:
dataset = TweetsDataset(train_data_filtered)

# Split into training and test
train_size = int(0.7 * len(dataset))
test_size = len(dataset) - train_size
trainset, testset = random_split(dataset, [train_size, test_size])

# # Dataloaders
trainloader = DataLoader(trainset, batch_size=1024, shuffle=True, num_workers=12)
testloader = DataLoader(testset, batch_size=1024, shuffle=False, num_workers=12)

lr_monitor = LearningRateMonitor(logging_interval='step')

# train(train_data_filtered, 3)
trainer = pl.Trainer(logger=logger, default_root_dir="checkpoints/", max_epochs=500)#, callbacks=[lr_monitor])
trainer.fit(model=model, train_dataloaders=trainloader, val_dataloaders=testloader)
# trainer.save

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type       | Params
------------------------------------
0 | mlp  | Sequential | 1.6 K 
------------------------------------
1.6 K     Trainable params
0         Non-trainable params
1.6 K     Total params
0.006     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/users/eleves-a/2020/danilo.marinho-fernandes/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:653: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
